In [2]:
import numpy as np
import pandas as pd
from numpy.random import rand
import matplotlib.pyplot as plt
%matplotlib inline
import itertools

In [3]:
import keras
from keras import backend as K
from keras import regularizers
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Activation,Dense, Dropout, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D
from keras.models import Sequential,load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

Using TensorFlow backend.


In [4]:
train="Train/"
test="Test/"
datagen = ImageDataGenerator(rescale=1./255)
train_data = datagen.flow_from_directory(directory=train, target_size=(256, 256), classes = ['Human', 'Not Human'], color_mode='rgb', class_mode='binary', batch_size=10, interpolation='nearest')
test_data = datagen.flow_from_directory(directory=test, target_size=(256, 256), classes = ['Human', 'Not Human'], color_mode='rgb', class_mode='binary', batch_size=10, interpolation='nearest')

Found 360 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [5]:
model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3))

In [6]:
out = model.output
out = GlobalAveragePooling2D()(out)
out = Dense(512, activation='relu')(out)
out = Dense(512, activation='relu')(out)
total_classes = 2
predictions = Dense(total_classes, activation='softmax')(out)
mymodel = Model(inputs=model.input, outputs=predictions)

In [7]:
mymodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [8]:
for layer in model.layers:
    layer.trainable = False #freeze model

In [9]:
mymodel.compile(Adam(lr=.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
mymodel.fit_generator(train_data,
                  steps_per_epoch=8,
                  validation_data=test_data,
                  validation_steps=8,
                  epochs=10,
                  verbose=2)

Epoch 1/10
 - 10s - loss: 0.7016 - acc: 0.5750 - val_loss: 0.6422 - val_acc: 0.8500
Epoch 2/10
 - 8s - loss: 0.6295 - acc: 0.8375 - val_loss: 0.6029 - val_acc: 0.8500
Epoch 3/10
 - 8s - loss: 0.5620 - acc: 0.9000 - val_loss: 0.5720 - val_acc: 0.8500
Epoch 4/10
 - 8s - loss: 0.5403 - acc: 0.9375 - val_loss: 0.5307 - val_acc: 0.9000
Epoch 5/10
 - 8s - loss: 0.5015 - acc: 0.8500 - val_loss: 0.4995 - val_acc: 0.8500
Epoch 6/10
 - 8s - loss: 0.4357 - acc: 0.9500 - val_loss: 0.4725 - val_acc: 0.8000
Epoch 7/10
 - 8s - loss: 0.4084 - acc: 0.9125 - val_loss: 0.4671 - val_acc: 0.8500
Epoch 8/10
 - 8s - loss: 0.3738 - acc: 0.9000 - val_loss: 0.4075 - val_acc: 0.8500
Epoch 9/10
 - 8s - loss: 0.3562 - acc: 0.9250 - val_loss: 0.4063 - val_acc: 0.8000
Epoch 10/10
 - 8s - loss: 0.3103 - acc: 0.9125 - val_loss: 0.3601 - val_acc: 0.8500
